In [1]:
# AADC basic "Hello World" example
# This example demonstrates how to use the AADC library to record a simple function and calculate its derivatives
import aadc
import numpy as np
import math

bind_swig2pybind_linearalgebra_i
Adding extend interface from SWIG to Pybind11 for class FixedLocalVolSurface
Adding extend interface from SWIG to Pybind11 for class GridModelLocalVolSurface


In [2]:
# Override defalt math.exp with aadc.math.exp to hand active types
math.exp = aadc.math.exp

In [3]:
# Simple function to record
# The function can take arbitrary input types and 
# can be defined in external file
def F(x,y,z):
    return math.exp(x / y + z)

In [4]:
# Object to hold recorded AADC Kernels that
# allow to speed up the calculation of the function itself
# and AAD for its derivatives
funcs = aadc.Functions()

In [5]:
# Define function arguments.
# Note you need to use aadc.idouble instead of float
x = aadc.idouble(1.0)
y = aadc.idouble(2.0)
z = aadc.idouble(3.0)

In [6]:
# Trigger recording of the function
funcs.start_recording()

You are using evaluation version of AADC. Expire date is 20240901


In [7]:
# Mark the function arguments as input and save reference argument ids
xArg = x.mark_as_input()
yArg = y.mark_as_input()
zArg = z.mark_as_input()

In [8]:
# Call the function and add some operations
f = F(x,y,z) + x

In [9]:
# Mark the result as output
fRes = f.mark_as_output()

In [10]:
# Stop recording
# After this step AADC Kernel containing native machine CPU code
# will be generated and stored in the funcs object
funcs.stop_recording()

In [11]:
# Check if the function is recorded properly
# and can be used for arbitrary input values
# This should return 0 if everything is OK, indicating
# that no branches in the function are not supported
funcs.print_passive_extract_locations()

Number active to passive conversions: 0 while recording Python


In [12]:
# New samples of input values to calculate the function and its derivatives at
# Note that the x input is a vector of 20 samples
# and the y and z are scalars
inputs = {
    xArg:(1.0 * np.random.normal(1, 0.2, 20)),
    yArg:(2.0),
    zArg:(3.0),
}

In [13]:
# Key: what output, value: what gradients are needed

request = {fRes:[xArg,yArg,zArg]}  

In [14]:
# Run AADC Kernel for array inputs using 4 CPU threads and avx2
Res = aadc.evaluate(funcs, request, inputs, aadc.ThreadPool(4))

In [15]:
# Function output
Res[0][fRes]

array([33.24205433, 34.6915409 , 37.52129798, 37.69674374, 32.36243554,
       35.50886523, 44.42939322, 32.24723311, 39.13724023, 31.05824264,
       37.46866464, 31.34508762, 29.13052269, 34.96900331, 34.22029599,
       34.67305414, 35.39841509, 43.32715   , 33.4486223 , 32.64016329])

In [16]:
print("df/dx")
Res[1][fRes][xArg]

df/dx


array([17.14619553, 17.82949053, 19.16784835, 19.25100457, 16.7323514 ,
       18.21547665, 22.45556536, 16.67819827, 19.93448792, 16.11995851,
       19.14290554, 16.25452073, 15.21763133, 17.96046918, 17.60716649,
       17.82076571, 18.1632877 , 21.92927875, 17.24347281, 16.86294837])

In [18]:
print("df/dy")
print(Res[1][fRes][yArg])


df/dy
[ -7.66672437  -8.68872797 -10.7699121  -10.90255339  -7.06172349
  -9.27838383 -16.28759015  -6.98335615 -12.00696843  -6.18652467
 -10.7302003   -6.37674175  -4.94247543  -8.88783664  -8.3530975
  -8.67550096  -9.19814652 -15.36829103  -7.81048443  -7.2514816 ]
